# 필요한 모듈 임포트

In [3]:
pip install selenium==4.8.2


Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install webdriver_manager

  Using cached webdriver_manager-3.8.6-py2.py3-none-any.whl (27 kB)
  Using cached python_dotenv-1.0.0-py3-none-any.whl (19 kB)
Note: you may need to restart the kernel to use updated packages.


3.141.0
3.8.6


In [16]:
from bs4 import BeautifulSoup
import selenium
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import numpy as np
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import re
print(selenium.__version__)

3.141.0


# Selenium을 이용하여 브라우저 열기, 클릭, 페이지 이동, 닫기

In [18]:
# ChromeOptions 객체를 생성하고 이를 options에 저장합니다. ChromeOptions 객체는 웹 드라이버에 대한 다양한 설정을 관리하는데 사용됩니다. 이를 통해 예를 들어 헤드리스 모드로 실행하거나 특정 확장 프로그램을 로드하는 등의 설정을 할 수 있습니다.
options = webdriver.ChromeOptions()

# # Selenium이 접근하기 어려운 웹페이지에 접근하게 해줌
# options.add_argument('--no-sandbox')
# # Shared memory를 사용하게 함
# options.add_argument('--disable-dev-shm-usage')
# WebDriver는 웹 페이지를 자동화하기 위한 인터페이스를 제공하는 클래스입니다. webdriver.Chrome()는 Chrome 브라우저를 컨트롤하기 위한 WebDriver 객체를 생성합니다. 이때, 첫번째 인자로 ChromeDriver의 경로를 받습니다. 이 경로를 자동으로 관리해주는 것이 ChromeDriverManager().install()입니다. ChromeDriverManager는 ChromeDriver의 최신 버전을 자동으로 다운로드하고 설치해주며, 그 경로를 반환해줍니다. 또한, options=options로 설정한 ChromeOptions 객체를 전달하여 웹 드라이버의 설정을 변경합니다.
driver = webdriver.Chrome(ChromeDriverManager().install(), options=options)


# options = webdriver.ChromeOptions()
# driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options) # 코드 돌릴때마다 자동으로 설정해줘서 오류가 잘 안 뜸

In [19]:
# 네이버 웹툰 - 토요일
url = 'https://comic.naver.com/webtoon?tab=sat' 
driver.get(url)
driver.maximize_window() # 브라우저의 크기 전체화면으로 확대

In [20]:
sunday_webtoon_button = driver.find_element(By.XPATH, '//*[@id="wrap"]/header/div[3]/nav/ul/li[8]/a') # 일요일 웹툰 버튼 클릭
sunday_webtoon_button.click()

In [21]:
driver.back() # 앞의 페이지로 이동

In [22]:
driver.forward() # 뒤의 페이지로 이동

In [23]:
driver.close() # 브라우저 창 닫기

# 네이버 토, 일 웹툰 정보 크롤링

In [30]:
options = webdriver.ChromeOptions()
driver = webdriver.Chrome(ChromeDriverManager().install(), options=options)
webtoon_list = [] # 토요일 일요일 데이터 프레임을 담을 리스트
weekend_dic = {0:'토요일', 1:'일요일'}
url = 'https://comic.naver.com/webtoon?tab=sat' # 우리가 갈 url
driver.get(url) 
driver.implicitly_wait(2) # 로딩이 잘 될때까지 기다림

########## 옵션임, window창을 띄워주는 역할
try:
    driver.maximize_window()
except:
    pass
##########
for weekend in range(2):
    # 페이지를 가장 아래까지 스크롤하는 방법

    interval = 1 # 1초에 한번씩 스크롤 내림

############### 스크롤, 자동으로 페이지 맨 아래까지 스크롤
    # 현재 문서 높이를 가져와서 저장 (현재 웹페이지의 전체 높이(스크롤 가능한 높이)를 픽셀 단위로 반환
    # document.body.scrollHeight (자바스크립트 코드)는 문서의 전체 높이를 나타낸다. 
    # 스크롤이 가능한 경우, 이 값은 보이는 viewport의 높이보다 클 수 있다.
    prev_height = driver.execute_script("return document.body.scrollHeight")
    
    # 반복 수행
    while True:
        # 스크롤을 가장 아래로 내림
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")
        # time.sleep(interval)
        # driver.execute_script("window.scrollTo(0, 0)")
        # break
        
        # 페이지 로딩 대기
        time.sleep(interval)

        # 현재 문서 높이를 가져와서 저장
        curr_height = driver.execute_script("return document.body.scrollHeight")
        if curr_height == prev_height:
            break

        prev_height = curr_height
###############


    # 해당 Xpath 정보 (개발자 창 열고, 마우스로 지정하고, 우클릭, copy)를 확인할때까지 5초간 기다림, 5초후에도 해당 Xpath를 찾지 못하면 에러 반환
    # elem = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, '//*[@id="content"]/div[1]/ul/li[40]/a/div/img')))

    columns = ['weekend', 'name', 'author', 'rate']         # 웹툰이름, 웹툰작가, 웹툰평점, 정보 크롤링
    values = []                                             # 토요일 웹툰 리스트와 일요일 웹툰 리스트 정보를 담을 빈 리스트 생성
    soup = BeautifulSoup(driver.page_source, 'lxml')        # 현재 브라우저의 page의 html 정보를 lxml을 이용하여 가져오기 ("driver.page_source" 이 쪽에 페이지소스를 넣는거)
    
    data_rows = soup.find_all('li', attrs={'class':'item'}) # 해당 조건에 맞는 정보들을 list로 반환 (웹툰 사이트의 각 웹툰들 <div class="component_wrap"> 부분

    for i, row in enumerate(data_rows):                     # data_rows 리스트에 있는 값들과 인덱스를 차례대로 가져온다

        print('{}번째 웹툰정보 크롤링'.format(i+1))
        blank = []
        blank.append(weekend_dic[weekend]) # 여기 날짜 부분은 웹툰 페이지에 명시 안되어있어서 따로 써준거

        name = row.find('span', attrs={'class':'ContentTitle__title--e3qXt'})       # 웹툰이름 가져오기
        if name:                                                                    # 해당 조건에 맞는 정보가 하나라도 있을 경우:
            name = name.get_text().strip()                                          # html에서 텍스트만 가져온 뒤 공백 제거
            blank.append(name)
        else:                                                                       # 해당 조건에 맞는 정보가 하나도 없을 경우:
            blank.append('Something is wrong')
            print('{}번째 웹툰이름 가져올때 문제발생'.format(i+1))
            continue


        author = row.find('a', attrs={'class':'ContentAuthor__author--CTAAP'})      # 웹툰작가 가져오기
        if author:
            author = author.get_text().strip()
            blank.append(author)
        else:
            blank.append('Something is wrong')
            print('{}번째 웹툰작가 가져올때 문제발생'.format(i+1))
            continue


        rate = row.find('span', attrs={'class':'Rating__star_area--dFzsb'})        # 웹툰평점 가져오기
        if rate:
            rate = rate.find('span', attrs={'class':'text'}).get_text().strip()    # class가 Rating__star_area--dFzsb인 span 태그 안에 있는 span 태그에서 텍스트 가져오기
            blank.append(rate)
        else:
            blank.append('Something is wrong')
            print('{}번째 웹툰평점 가져올때 문제발생'.format(i+1))
            continue


#         image_url = row.find('img', attrs={'class':'Poster__image--d9XTI'})     # 웹툰이름 가져오기
#         if image_url:                                                           # 해당 조건에 맞는 정보가 하나라도 있을 경우
#             image_url = image_url['src']                                        # src attribute의 값을 가져온다
#             if image_url.startswith('//'):                                      # image_url이 https: 로 시작하지 않는 경우
#                 image_url = 'https:' + image_url
#             blank.append(image_url)
#         else:
#             blank.append('Something is wrong')
#             print('{}번째 웹툰이미지 가져올때 문제발생'.format(i+1))
#             continue

        values.append(blank)
        print('---------------------------------------------------')

    sunday_webtoon_button = driver.find_element(By.XPATH, '//*[@id="wrap"]/header/div[3]/nav/ul/li[8]/a') # 일요일 웹툰 클릭 (XPATH가 변동이 적음, 그래서 XPATH로 가져옴)
    
    sunday_webtoon_button.click()
    time.sleep(1)

    df = pd.DataFrame(values, columns = columns)
    webtoon_list.append(df) # 최종적으로 [[토요일 웹툰 정보 리스트들], [일요을 웹툰 정보 리스트들]] 형태로 반환됨


1번째 웹툰정보 크롤링
---------------------------------------------------
2번째 웹툰정보 크롤링
---------------------------------------------------
3번째 웹툰정보 크롤링
---------------------------------------------------
4번째 웹툰정보 크롤링
---------------------------------------------------
5번째 웹툰정보 크롤링
---------------------------------------------------
6번째 웹툰정보 크롤링
---------------------------------------------------
7번째 웹툰정보 크롤링
---------------------------------------------------
8번째 웹툰정보 크롤링
---------------------------------------------------
9번째 웹툰정보 크롤링
---------------------------------------------------
10번째 웹툰정보 크롤링
---------------------------------------------------
11번째 웹툰정보 크롤링
---------------------------------------------------
12번째 웹툰정보 크롤링
---------------------------------------------------
13번째 웹툰정보 크롤링
---------------------------------------------------
14번째 웹툰정보 크롤링
---------------------------------------------------
15번째 웹툰정보 크롤링
---------------------------------------------------
16번째 웹툰정보 크롤링
-----

AttributeError: module 'pandas' has no attribute 'DataFrame'

In [22]:
df_ = pd.concat(webtoon_list).reset_index(drop=True) # index 초기화 해줘야함
df_.head(10)

,weekend,name,author,rate
0,토요일,호랑이형님,이상규,9.92
1,토요일,99강화나무몽둥이,홍실 / 지페리,9.81
2,토요일,취사병 전설이 되다,제이로빈 / 이진수,9.87
3,토요일,물위의 우리,뱁새 / 왈패,9.98
4,토요일,프리드로우,전선욱,9.79
5,토요일,망나니 소교주로 환생했다,재무 / 전마두 / 대은호,9.85
6,토요일,마도전생기,포스스튜디오 / codezero,9.84
7,토요일,은탄,김규삼,9.96
8,토요일,귀환했는데 입대 전날이다,해일 / 오코믹 / 황금비둘기,9.90
9,토요일,작전명 순정,꼬까리 / 들덤,9.96


In [23]:
df_.shape

(160, 4)

# Dataframe을 csv 파일로 저장

In [24]:
df_.to_csv('네이버웹툰_토일_크롤링.csv', encoding = 'utf-8-sig') # 한글이 들어간 정보는 글자가 깨질 수 있어 encoding을 다음과 같이 지정하여 저장 (한글은 utf-8로 안해주면 깨짐)